# Load data

In [ ]:
from math import isnan

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import wandb

dataset_match = {
    "fbirn": "FBIRN ICA",
    "fbirn_roi": "FBIRN Schaefer 200 ROI",
    "bsnip": "BSNIP ICA",
    "cobre": "COBRE ICA",

    "abide": "ABIDE1 ICA 569 subjects",
    "abide_869": "ABIDE1 ICA",
    "abide_roi": "ABIDE1 Schaefer 200 ROI",

    "oasis": "OASIS3 ICA",
    "adni": "ADNI ICA",

    "hcp": "HCP ICA",
    "hcp_roi": "HCP Schaefer 200 ROI",
    "hcp_non_mni": "HCP Deskian/Killiany ROI Old",
    "hcp_non_mni_2": "HCP Deskian/Killiany ROI",
    "hcp_mni": "HCP Deskian/Killiany ROI MNI Oldest",
    "hcp_mni_2": "HCP Deskian/Killiany ROI MNI Old",
    "hcp_mni_3": "HCP Deskian/Killiany ROI MNI",

    "ukb": "UK Biobank (Sex) ICA",
    "ukb_age_bins": "UK Biobank (Age-Sex) ICA",

    "time_fbirn": "FBIRN (Time Direction) ICA",

    "fbirn_bsnip": "FxB",
    "fbirn_cobre": "FxC",

    "bsnip_fbirn": "BxF",
    "bsnip_cobre": "BxC",

    "cobre_fbirn": "CxF",
    "cobre_bsnip": "CxB",

    "oasis_adni": "OxA",
    "adni_oasis": "AxO",
}

model_match = {
    "mlp": "MLP",
    "lstm": "LSTM",
    "mean_lstm": r"$\mu$LSTM",
    "transformer": "Transformer",
    "mean_transformer": r"$\mu$Transformer",
    "milc": "MILC",
    "stdim": "ST-DIM",
    "dice": "DICE",
    "bnt": "BNT",
    "fbnetgen": "FBNetGen",
    "brainnetcnn": "BrainNetCNN",
    "lr": "LR",
}

def fix_run(run, dataset):
    for _, row in run.history().iterrows():
        if f"{dataset}score" in row and f"{dataset}accuracy" in row and "training_time" in row and f"{dataset}average_time" in row:
            if row[f"{dataset}score"] is not None and row[f"{dataset}accuracy"] is not None and row["training_time"] is not None and row[f"{dataset}average_time"] is not None:
                if not isnan(row[f"{dataset}score"]) and not isnan(row[f"{dataset}accuracy"]) and not isnan(row["training_time"]) and not isnan(row[f"{dataset}average_time"]):
                    auc = row[f"{dataset}score"]
                    acc = row[f"{dataset}accuracy"]
                    train_t = row["training_time"]
                    inf_t= row[f"{dataset}average_time"]
                    break
    
    run.summary[f"{dataset}score"] = auc
    run.summary[f"{dataset}accuracy"] = acc
    run.summary["training_time"] = train_t
    run.summary[f"{dataset}average_time"] = inf_t

    run.summary.update()

    return auc, acc, train_t, inf_t

def load_run(proj_name, dataset):
    if dataset is not None:
        dataset += "_"
    else:
        dataset = "test_"

    api = wandb.Api(timeout=19)
    # Project is specified by <entity/project-name>
    runs = api.runs(f"pavalipopov/{proj_name}")

    summary_list = []
    runs_list = []
    config_list = []
    for i, run in enumerate(runs): 
        # print(f"Run {i}")
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)
        config_list.append(run.config)
        runs_list.append(run)


    AUC_score = []
    accuracy = []
    train_time = []
    inference_time = []
    params = []
    shuffling = []

    for i, summary in enumerate(summary_list):
        everything_is_cool = True
        everything_is_cool = everything_is_cool and f"{dataset}score" in summary and summary[f"{dataset}score"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}accuracy" in summary and summary[f"{dataset}accuracy"] is not None
        everything_is_cool = everything_is_cool and "training_time" in summary and summary["training_time"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}average_time" in summary and summary[f"{dataset}average_time"] is not None

        if everything_is_cool:
            AUC_score.append(summary[f"{dataset}score"])
            accuracy.append(summary[f"{dataset}accuracy"])
            train_time.append(summary["training_time"])
            inference_time.append(summary[f"{dataset}average_time"])
        else:
            print(f"Run {i} summary is broken, fixing it")
            auc, acc, train_t, inf_t = fix_run(runs_list[i], dataset)
            AUC_score.append(auc)
            accuracy.append(acc)
            train_time.append(train_t)
            inference_time.append(inf_t)

        if "params" in summary:
            params.append(summary["params"])
        else:
            params.append(None)

        if "permute" in config_list[i]["general"]:
            if config_list[i]["general"]["permute"] == False:
                shuffling.append("None")
            else:
                shuffling.append(config_list[i]["general"]["permute"])
        else:
            shuffling.append("None")
    
    return AUC_score, accuracy, train_time, inference_time, params, shuffling

def load_metrics(paths_dict, ds_dict, model_dict):
    data_list = []

    for model_name in paths_dict:
        print(model_name)
        for dataset_name in paths_dict[model_name]:
            print("\t ", dataset_name)
            
            if len(paths_dict[model_name][dataset_name]) == 2:
                path = paths_dict[model_name][dataset_name][0]
                dataset = paths_dict[model_name][dataset_name][1]
            elif len(paths_dict[model_name][dataset_name]) == 1:
                path = paths_dict[model_name][dataset_name][0]
                dataset = "test"
            else:
                raise NotImplementedError()

            print("\t\t sub_dataset:", dataset)

            AUC_score, accuracy, train_time, inference_time, params, shuffling = load_run(path, dataset)
            
            if dataset != "test":
                dataset = dataset_name + "_" + dataset
            else:
                dataset = dataset_name
            length = len(AUC_score)

            data_list.append(
                pd.DataFrame(
                    {
                        "Model": [model_dict[model_name]]*length,
                        "Dataset": [ds_dict[dataset]]*length,
                        "AUROC": AUC_score,
                        "Accuracy": accuracy,
                        "Train time": train_time,
                        "Inference time": inference_time,
                        "Trainable params": params,
                        "Shuffling": shuffling,
                    }
                )
            )
    
    return pd.concat(data_list)

In [ ]:
projects = {
    "mlp": {
        "hcp": ("all_mni-exp-mlp_defHP-hcp", ),
        "hcp_roi": ("all_mni-exp-mlp_defHP-hcp_roi", ),
        "hcp_non_mni_2": ("all_mni-exp-mlp_defHP-hcp_non_mni_2", ),
        "hcp_mni_3": ("all_mni-exp-mlp_defHP-hcp_mni_3", ),
    },
    "lstm": {
        "hcp": ("all_mni-exp-lstm_defHP-hcp", ),
        "hcp_roi": ("all_mni-exp-lstm_defHP-hcp_roi", ),
        "hcp_non_mni_2": ("all_mni-exp-lstm_defHP-hcp_non_mni_2", ),
        "hcp_mni_3": ("all_mni-exp-lstm_defHP-hcp_mni_3", ),
    },
    "mean_lstm": {
        "hcp": ("all_mni-exp-mean_lstm_defHP-hcp", ),
        "hcp_roi": ("all_mni-exp-mean_lstm_defHP-hcp_roi", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_lstm_defHP-hcp_non_mni_2", ),
        "hcp_mni_3": ("all_mni-exp-mean_lstm_defHP-hcp_mni_3", ),
    },
    "transformer": {
        "hcp": ("all_mni-exp-transformer_defHP-hcp", ),
        "hcp_roi": ("all_mni-exp-transformer_defHP-hcp_roi", ),
        "hcp_non_mni_2": ("all_mni-exp-transformer_defHP-hcp_non_mni_2", ),
        "hcp_mni_3": ("all_mni-exp-transformer_defHP-hcp_mni_3", ),
    },
    "mean_transformer": {
        "hcp": ("all_mni-exp-mean_transformer_defHP-hcp", ),
        "hcp_roi": ("all_mni-exp-mean_transformer_defHP-hcp_roi", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_transformer_defHP-hcp_non_mni_2", ),
        "hcp_mni_3": ("all_mni-exp-mean_transformer_defHP-hcp_mni_3", ),
    },
}

projects_single = {
    "mlp": {
        "hcp": ("all_mni-exp-mlp_defHP-hcp-perm_Single", ),
        "hcp_roi": ("all_mni-exp-mlp_defHP-hcp_roi-perm_Single", ),
        "hcp_non_mni_2": ("all_mni-exp-mlp_defHP-hcp_non_mni_2-perm_Single", ),
        "hcp_mni_3": ("all_mni-exp-mlp_defHP-hcp_mni_3-perm_Single", ),
    },
    "lstm": {
        "hcp": ("all_mni-exp-lstm_defHP-hcp-perm_Single", ),
        "hcp_roi": ("all_mni-exp-lstm_defHP-hcp_roi-perm_Single", ),
        "hcp_non_mni_2": ("all_mni-exp-lstm_defHP-hcp_non_mni_2-perm_Single", ),
        "hcp_mni_3": ("all_mni-exp-lstm_defHP-hcp_mni_3-perm_Single", ),
    },
    "mean_lstm": {
        "hcp": ("all_mni-exp-mean_lstm_defHP-hcp-perm_Single", ),
        "hcp_roi": ("all_mni-exp-mean_lstm_defHP-hcp_roi-perm_Single", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_lstm_defHP-hcp_non_mni_2-perm_Single", ),
        "hcp_mni_3": ("all_mni-exp-mean_lstm_defHP-hcp_mni_3-perm_Single", ),
    },
    "transformer": {
        "hcp": ("all_mni-exp-transformer_defHP-hcp-perm_Single", ),
        "hcp_roi": ("all_mni-exp-transformer_defHP-hcp_roi-perm_Single", ),
        "hcp_non_mni_2": ("all_mni-exp-transformer_defHP-hcp_non_mni_2-perm_Single", ),
        "hcp_mni_3": ("all_mni-exp-transformer_defHP-hcp_mni_3-perm_Single", ),
    },
    "mean_transformer": {
        "hcp": ("all_mni-exp-mean_transformer_defHP-hcp-perm_Single", ),
        "hcp_roi": ("all_mni-exp-mean_transformer_defHP-hcp_roi-perm_Single", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_transformer_defHP-hcp_non_mni_2-perm_Single", ),
        "hcp_mni_3": ("all_mni-exp-mean_transformer_defHP-hcp_mni_3-perm_Single", ),
    },
}

projects_multiple = {
    "mlp": {
        "hcp": ("all_mni-exp-mlp_defHP-hcp-perm_Multiple", ),
        "hcp_roi": ("all_mni-exp-mlp_defHP-hcp_roi-perm_Multiple", ),
        "hcp_non_mni_2": ("all_mni-exp-mlp_defHP-hcp_non_mni_2-perm_Multiple", ),
        "hcp_mni_3": ("all_mni-exp-mlp_defHP-hcp_mni_3-perm_Multiple", ),
    },
    "lstm": {
        "hcp": ("all_mni-exp-lstm_defHP-hcp-perm_Multiple", ),
        "hcp_roi": ("all_mni-exp-lstm_defHP-hcp_roi-perm_Multiple", ),
        "hcp_non_mni_2": ("all_mni-exp-lstm_defHP-hcp_non_mni_2-perm_Multiple", ),
        "hcp_mni_3": ("all_mni-exp-lstm_defHP-hcp_mni_3-perm_Multiple", ),
    },
    "mean_lstm": {
        "hcp": ("all_mni-exp-mean_lstm_defHP-hcp-perm_Multiple", ),
        "hcp_roi": ("all_mni-exp-mean_lstm_defHP-hcp_roi-perm_Multiple", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_lstm_defHP-hcp_non_mni_2-perm_Multiple", ),
        "hcp_mni_3": ("all_mni-exp-mean_lstm_defHP-hcp_mni_3-perm_Multiple", ),
    },
    "transformer": {
        "hcp": ("all_mni-exp-transformer_defHP-hcp-perm_Multiple", ),
        "hcp_roi": ("all_mni-exp-transformer_defHP-hcp_roi-perm_Multiple", ),
        "hcp_non_mni_2": ("all_mni-exp-transformer_defHP-hcp_non_mni_2-perm_Multiple", ),
        "hcp_mni_3": ("all_mni-exp-transformer_defHP-hcp_mni_3-perm_Multiple", ),
    },
    "mean_transformer": {
        "hcp": ("all_mni-exp-mean_transformer_defHP-hcp-perm_Multiple", ),
        "hcp_roi": ("all_mni-exp-mean_transformer_defHP-hcp_roi-perm_Multiple", ),
        "hcp_non_mni_2": ("all_mni-exp-mean_transformer_defHP-hcp_non_mni_2-perm_Multiple", ),
        "hcp_mni_3": ("all_mni-exp-mean_transformer_defHP-hcp_mni_3-perm_Multiple", ),
    },
}

In [ ]:
data_n = load_metrics(projects, dataset_match, model_match)
data_s = load_metrics(projects_single, dataset_match, model_match)
data_m = load_metrics(projects_multiple, dataset_match, model_match)

data = pd.concat([data_n, data_s, data_m])

# Plot data

## All results

In [ ]:
import matplotlib.ticker as ticker
from statannot import add_stat_annotation

import matplotlib.pyplot as plt

sns.reset_orig()
sns.set_theme(
    style="whitegrid", 
    font_scale = 1.5,
    rc={'figure.figsize':(9,4)},
)

plot_data = data.query(
        "Shuffling == 'None'")

models = [
    "MLP", 
    'LSTM', 
    r'$\mu$LSTM', 
    'Transformer', 
    r'$\mu$Transformer', 
    "MILC", 
    "ST-DIM", 
    "DICE", 
    "BNT", 
    "FBNetGen", 
    "BrainNetCNN", 
    "LR"
]
datasets = [
    "HCP Deskian/Killiany ROI",
    "HCP Deskian/Killiany ROI MNI",
    "HCP Schaefer 200 ROI",
    "HCP ICA",
]
hue_order = datasets
x_order = [
    "MLP", 
    r'$\mu$LSTM', 
    r'$\mu$Transformer', 
    'LSTM', 
    'Transformer'
]

palette = { item: plt.cm.tab20(i) for i, item in enumerate(datasets)}

ax = sns.boxplot(
    x="Model", 
    order=x_order,
    y="AUROC",
    hue="Dataset",
    hue_order=hue_order,
    data=plot_data,
    palette=palette,
    showfliers = False
)

ax.set(xlabel=None)
ax.axhline(0.5)

plt.yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
# plt.legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', borderaxespad=0)
plt.legend(bbox_to_anchor=(0.01, -0.15), loc='upper left', borderaxespad=0)
# plt.title(label="ICA - Neuromark")

plt.ylim(0.45, 1)
# ax.set_xlim(xmin=-0.5, xmax=8.5)

add_stat_annotation(ax, data=plot_data, x="Model", order=x_order, y="AUROC", hue="Dataset", hue_order=hue_order,
                    box_pairs=[
                        (("MLP", "HCP Deskian/Killiany ROI"), ("MLP", "HCP Deskian/Killiany ROI MNI")),
                        (("LSTM", "HCP Deskian/Killiany ROI"), ("LSTM", "HCP Deskian/Killiany ROI MNI")),
                        ((r'$\mu$LSTM', "HCP Deskian/Killiany ROI"), (r'$\mu$LSTM', "HCP Deskian/Killiany ROI MNI")),
                        (("Transformer", "HCP Deskian/Killiany ROI"), ("Transformer", "HCP Deskian/Killiany ROI MNI")),
                        ((r'$\mu$Transformer', "HCP Deskian/Killiany ROI"), (r'$\mu$Transformer', "HCP Deskian/Killiany ROI MNI")),
                    ],
                    test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
                    pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
                    line_offset=0.01, line_offset_to_box=0.01, line_height=0.01, fontsize=24)

# plt.show()
plt.savefig(
    "comparison.png",
    format="png",
    dpi=300,
    bbox_inches='tight',
)
# plt.savefig(
#     "comparison.svg",
#     format="svg",
#     # dpi=300,
#     bbox_inches='tight',
# )
plt.close()

## Shuffling results

In [ ]:
import matplotlib.ticker as ticker
from statannot import add_stat_annotation

sns.reset_orig()
sns.set_theme(
    style="whitegrid", 
    font_scale = 1.5,
)

models = [
    "MLP", 
    'LSTM', 
    r'$\mu$LSTM', 
    'Transformer', 
    r'$\mu$Transformer', 
    "MILC", 
    "ST-DIM", 
    "DICE", 
    "BNT", 
    "FBNetGen", 
    "BrainNetCNN", 
    "LR"
]

shuffling = ["False", "True"]
palette = { item: plt.cm.tab20(i) for i, item in enumerate(shuffling)}
hue_order = shuffling

x_order = [
    "MLP", 
    r'$\mu$LSTM', 
    r'$\mu$Transformer', 
    'LSTM', 
    'Transformer'
]

datasets = [
    "HCP Deskian/Killiany ROI",
    "HCP Deskian/Killiany ROI MNI",
    "HCP Schaefer 200 ROI",
    "HCP ICA",
]

fig = plt.figure(constrained_layout=True, figsize=(9*len(datasets)//2, 4*len(datasets)//2))
axs = fig.subplots(len(datasets)//2, len(datasets)//2)

sns.set_theme(
    style="whitegrid", 
    font_scale = 2, 
)

for i, dataset in enumerate(datasets):
    plot_data = data.query(
        "Dataset == @dataset")
    plot_data = plot_data.query(
        "Shuffling in ['None', 'Multiple']")

    plot_data = plot_data.replace('None', 'False')
    plot_data = plot_data.replace('Multiple', 'True')

    sns.boxplot(
        x="Model", 
        order=x_order,
        y="AUROC",
        hue="Shuffling",
        hue_order=hue_order,
        data=plot_data,
        palette=palette,
        showfliers = False,
        ax = axs[i//2][i%2]
    )
    

    axs[i//2][i%2].set_yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
    # subfigs[i].axhline(0.5)
    if i%2 == 1:
        axs[i//2][i%2].set(yticklabels=[])
        axs[i//2][i%2].set(ylabel=None)

    if i%2 == 1 and i//2 == 0:
        axs[i//2][i%2].legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', borderaxespad=0, title="Shuffling")
    else:
        axs[i//2][i%2].legend([],[], frameon=False)
        
    axs[i//2][i%2].set_title(dataset, fontsize=16)

    axs[i//2][i%2].set_ylim(0.45, 1.0)
    axs[i//2][i%2].axhline(0.5)
    axs[i//2][i%2].set(xlabel=None)

    # try:
    #     add_stat_annotation(axs[i], data=plot_data, x="Model", y="AUC", hue="Permuted",
    #             box_pairs=[
    #                 (("MLP", True), ("MLP", False)),
    #                 (("LSTM", True), ("LSTM", False)),
    #                 (("Mean LSTM", True), ("Mean LSTM", False)),
    #                 (("TF", True), ("TF", False)),
    #                 (("Mean TF", True), ("Mean TF", False)),
    #             ],
    #             test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
    #             pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
    #             line_offset=0.01, line_offset_to_box=0.02, line_height=0.01, fontsize=13)
    # except ValueError:
    #     pass

# plt.show()
plt.savefig(
    "./shuffling.png",
    format="png",
    dpi=300,
    bbox_inches='tight',
    transparent=False,
)
plt.close()